<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# FAST-OAD-GA Tutorial

FAST-OAD-GA is an add-on package for [FAST-OAD framework](https://github.com/fast-aircraft-design/FAST-OAD) for performing rapid Overall Aircraft Design in the category General Aviation (GA). The computational core of FAST-OAD being based on the  [OpenMDAO framework](https://openmdao.org/).

## 1. Setting up and analyzing the initial problem

To organize our work, we propose to use two user folders `data/` and `workdir/`. For instance, in `data/` we store a XML file which describes the Beechcraft Duchess. In `workdir/`, we store files generated or modified by FAST-OAD.

In [1]:
import os.path as pth
import os
import openmdao.api as om
from fastoad import api as api_cs25
from fastga.command import api as api_cs23
import logging
from fastoad.gui import VariableViewer
import shutil

# Define relative path
DATA_FOLDER_PATH = 'data'
WORK_FOLDER_PATH = 'workdir'

# Remove work folder
shutil.rmtree(WORK_FOLDER_PATH, ignore_errors=True)

# Define files
CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, 'oad_process.yml')
SOURCE_FILE = pth.join(DATA_FOLDER_PATH, 'beechcraft_76.xml')

# For having log messages on screen
logging.basicConfig(level=logging.INFO, format='%(levelname)-8s: %(message)s')

# For using all screen width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

Failed to import module fastga.models.loops.unitary_tests.test_wing_area.py
Failed to import module fastga.models.aerodynamics.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.propulsion.fuel_propulsion.basicIC_engine.basicIC_engineOM.py
Failed to import module fastga.models.weight.mass_breakdown.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.load_analysis.unitary_tests.test_maxwell_x57.py
Failed to import module fastga.models.propulsion.fuel_propulsion.basicIC_engine.unitary_tests.test_openmdao_engine.py
Failed to import module fastga.models.handling_qualities.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.performances.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.weight.cg.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.handling_qualities.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.aerodynamics.unitary_tests.test_slipstream_x57.p

After defining a configuration file name, we can ask FAST-GA to generate a default configuration file based on the default OAD model implemented in the framework:

In [2]:
api_cs23.generate_configuration_file(CONFIGURATION_FILE, overwrite=True)

INFO    : Sample configuration written in workdir\oad_process.yml


You can now checkout the generated [configuration file](./workdir/oad_process.toml). In this configuration file, we have specified an input file name 'problem_inputs.xml'. We can ask FAST-OAD to generate the inputs of the default model with the CeRAS parameters as default values:

In [3]:
api_cs25.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

INFO    : Loading bundles from D:/fl.lutz/FAST/FAST-OAD/FAST-GA/src/fastga
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_chords (ID 68 )
INFO    : Installed bundle fastga.models.load_analysis.loads (ID 121 )
INFO    : Installed bundle fastga.models.aerodynamics.components.cd0_fuselage (ID 19 )
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_distance (ID 69 )
INFO    : Installed bundle fastga.models.propulsion.propulsion (ID 147 )
INFO    : Installed bundle fastga.utils.physics.tests (ID 445 )
INFO    : Installed bundle fastga.models.aerodynamics.components.cd0_vt (ID 25 )
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_efficiency (ID 70 )
INFO    : Installed bundle fastga.models.aerodynamics.components.cd0_other (ID 23 )
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_mac (ID 71 )
INFO    : Installed bundle fastga.

INFO    : Installed bundle fastga.models.aerodynamics.aerodynamics_high_speed (ID 14 )
INFO    : Installed bundle fastga.models.load_analysis.unitary_tests.dummy_engines (ID 125 )
INFO    : Installed bundle fastga.utils (ID 443 )
INFO    : Installed bundle fastga.models.handling_qualities.unitary_tests (ID 414 )
INFO    : Installed bundle fastga.models.aerodynamics.aerodynamics_low_speed (ID 15 )
INFO    : Installed bundle fastga.models.aerodynamics.resources (ID 398 )
INFO    : Installed bundle fastga.models.geometry.geom_components.vt.components.compute_vt_chords (ID 79 )
INFO    : Installed bundle fastga.command.resources (ID 385 )
INFO    : Installed bundle fastga.models.geometry.geom_components.vt.components.compute_vt_mac (ID 80 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.unitary_tests.dummy_engines (ID 196 )
INFO    : Installed bundle fastga.models.geometry.geom_components.vt.components.compute_vt_sweep (ID 81 )
INFO    : Installed bundle fastga.notebooks.tu

INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a1_wing_cg (ID 150 )
INFO    : Installed bundle fastga.models.geometry.geom_components.wing.components (ID 409 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a2_fuselage_cg (ID 151 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.max_cg_ratio (ID 168 )
INFO    : Installed bundle fastga.utils.physics (ID 444 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing.compute_to_rotation_limit (ID 109 )
INFO    : Installed bundle fastga.models.propulsion.fuel_propulsion.basicIC_engine.unitary_tests (ID 424 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a5_landing_gear_cg (ID 154 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.ratio_aft (ID 170 )
INFO    : Installed bundle fastga.models.aerodynamics.external.openvsp.resources (ID 393 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_a

'D:\\fl.lutz\\FAST\\FAST-OAD\\FAST-GA\\src\\fastga\\notebooks\\tutorial\\workdir\\problem_inputs.xml'

You can now checkout the generated [input file](./workdir/problem_inputs.xml). As shown previously in the user file architecture, the values in this file can be modified by the user and will be considered by FAST-OAD when executing a computational process.

A useful feature that FAST-OAD provides is to list the outputs of the model defined in the configuration file:

In [4]:
api_cs25.list_variables(CONFIGURATION_FILE)

INFO    : Loading bundles from D:/fl.lutz/FAST/FAST-OAD/FAST-GA/src/fastga
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_chords (ID 68 )
INFO    : Installed bundle fastga.models.load_analysis.loads (ID 121 )
INFO    : Installed bundle fastga.models.aerodynamics.components.cd0_fuselage (ID 19 )
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_distance (ID 69 )
INFO    : Installed bundle fastga.models.propulsion.propulsion (ID 147 )
INFO    : Installed bundle fastga.utils.physics.tests (ID 445 )
INFO    : Installed bundle fastga.models.aerodynamics.components.cd0_vt (ID 25 )
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_efficiency (ID 70 )
INFO    : Installed bundle fastga.models.aerodynamics.components.cd0_other (ID 23 )
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_mac (ID 71 )
INFO    : Installed bundle fastga.

INFO    : Installed bundle fastga.models.aerodynamics.aerodynamics_high_speed (ID 14 )
INFO    : Installed bundle fastga.models.load_analysis.unitary_tests.dummy_engines (ID 125 )
INFO    : Installed bundle fastga.utils (ID 443 )
INFO    : Installed bundle fastga.models.handling_qualities.unitary_tests (ID 414 )
INFO    : Installed bundle fastga.models.aerodynamics.aerodynamics_low_speed (ID 15 )
INFO    : Installed bundle fastga.models.aerodynamics.resources (ID 398 )
INFO    : Installed bundle fastga.models.geometry.geom_components.vt.components.compute_vt_chords (ID 79 )
INFO    : Installed bundle fastga.command.resources (ID 385 )
INFO    : Installed bundle fastga.models.geometry.geom_components.vt.components.compute_vt_mac (ID 80 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.unitary_tests.dummy_engines (ID 196 )
INFO    : Installed bundle fastga.models.geometry.geom_components.vt.components.compute_vt_sweep (ID 81 )
INFO    : Installed bundle fastga.notebooks.tu

INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a1_wing_cg (ID 150 )
INFO    : Installed bundle fastga.models.geometry.geom_components.wing.components (ID 409 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a2_fuselage_cg (ID 151 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.max_cg_ratio (ID 168 )
INFO    : Installed bundle fastga.utils.physics (ID 444 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing.compute_to_rotation_limit (ID 109 )
INFO    : Installed bundle fastga.models.propulsion.fuel_propulsion.basicIC_engine.unitary_tests (ID 424 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a5_landing_gear_cg (ID 154 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.ratio_aft (ID 170 )
INFO    : Installed bundle fastga.models.aerodynamics.external.openvsp.resources (ID 393 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_a

NAME,I/O,DESCRIPTION
data:TLAR:NPAX_design,IN,
data:TLAR:luggage_mass_design,IN,
data:TLAR:range,IN,top-level requirement: design range
data:TLAR:v_approach,IN,
data:TLAR:v_cruise,IN,
data:TLAR:v_limit,IN,
data:TLAR:v_max_sl,IN,
data:geometry:cabin:aisle_width,IN,width of aisles
data:geometry:cabin:luggage:mass_max,IN,
data:geometry:cabin:seats:passenger:NPAX_max,IN,


Another useful feature is to list the modules of the model defined in the configuration file:

In [5]:
api_cs25.list_modules(CONFIGURATION_FILE)

INFO    : Loading bundles from D:/fl.lutz/FAST/FAST-OAD/FAST-GA/src/fastga
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_chords (ID 68 )
INFO    : Installed bundle fastga.models.load_analysis.loads (ID 121 )
INFO    : Installed bundle fastga.models.aerodynamics.components.cd0_fuselage (ID 19 )
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_distance (ID 69 )
INFO    : Installed bundle fastga.models.propulsion.propulsion (ID 147 )
INFO    : Installed bundle fastga.utils.physics.tests (ID 445 )
INFO    : Installed bundle fastga.models.aerodynamics.components.cd0_vt (ID 25 )
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_efficiency (ID 70 )
INFO    : Installed bundle fastga.models.aerodynamics.components.cd0_other (ID 23 )
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_mac (ID 71 )
INFO    : Installed bundle fastga.

INFO    : Installed bundle fastga.models.aerodynamics.aerodynamics_high_speed (ID 14 )
INFO    : Installed bundle fastga.models.load_analysis.unitary_tests.dummy_engines (ID 125 )
INFO    : Installed bundle fastga.utils (ID 443 )
INFO    : Installed bundle fastga.models.handling_qualities.unitary_tests (ID 414 )
INFO    : Installed bundle fastga.models.aerodynamics.aerodynamics_low_speed (ID 15 )
INFO    : Installed bundle fastga.models.aerodynamics.resources (ID 398 )
INFO    : Installed bundle fastga.models.geometry.geom_components.vt.components.compute_vt_chords (ID 79 )
INFO    : Installed bundle fastga.command.resources (ID 385 )
INFO    : Installed bundle fastga.models.geometry.geom_components.vt.components.compute_vt_mac (ID 80 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.unitary_tests.dummy_engines (ID 196 )
INFO    : Installed bundle fastga.models.geometry.geom_components.vt.components.compute_vt_sweep (ID 81 )
INFO    : Installed bundle fastga.notebooks.tu

INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a1_wing_cg (ID 150 )
INFO    : Installed bundle fastga.models.geometry.geom_components.wing.components (ID 409 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a2_fuselage_cg (ID 151 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.max_cg_ratio (ID 168 )
INFO    : Installed bundle fastga.utils.physics (ID 444 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing.compute_to_rotation_limit (ID 109 )
INFO    : Installed bundle fastga.models.propulsion.fuel_propulsion.basicIC_engine.unitary_tests (ID 424 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a5_landing_gear_cg (ID 154 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.ratio_aft (ID 170 )
INFO    : Installed bundle fastga.models.aerodynamics.external.openvsp.resources (ID 393 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_a

AVAILABLE MODULE IDENTIFIERS,MODULE PATH
fastga.aerodynamics.highspeed.legacy,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\aerodynamics\aerodynamics_high_speed.py
fastga.aerodynamics.lowspeed.legacy,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\aerodynamics\aerodynamics_low_speed.py
fastga.geometry.alternate,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\geometry\geometry.py
fastga.geometry.legacy,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\geometry\geometry.py
fastga.handling_qualities.all_handling_qualities,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\handling_qualities\handling_qualities.py
fastga.handling_qualities.static_margin,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\handling_qualities\compute_static_margin.py
fastga.handling_qualities.tail_sizing,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\handling_qualities\tail_sizing\update_tail_areas.py
fastga.loads.legacy,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\load_analysis\loads.py
fastga.loop.mtow,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\weight\mass_breakdown\update_mtow.py
fastga.loop.wing_area,D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\loops\update_wing_area.py


Another useful feature is the [N2 diagram](http://openmdao.org/twodocs/versions/latest/basic_guide/make_n2.html) visualization available in OpenMDAO to see the structure of the model:

In [6]:
N2_FILE = pth.join(WORK_FOLDER_PATH, 'n2.html')
api_cs25.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=N2_FILE, width='100%', height='500px')

INFO    : N2 diagram written in D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\notebooks\tutorial\workdir\n2.html


Alternatively, you can create a [WhatsOpt](https://github.com/OneraHub/WhatsOpt-Doc#whatsopt-documentation) account to generate the XDSM of the problem. If your account is created, you may uncomment next lines and run them (this should take ~ 1 min):

In [ ]:
XDSM_FILE = pth.join(WORK_FOLDER_PATH, 'xdsm.html')
api_cs25.write_xdsm(CONFIGURATION_FILE, XDSM_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=XDSM_FILE, width='100%', height='500px')

## 2. Running your first MDA

### Beechcraft
Here we run an MDA, that is solving the multidisciplinary couplings using the different nested solvers in the model, without running the optimization problem even if it is defined in the configuration file.

In [7]:
eval_problem = api_cs25.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

INFO    : Loading bundles from D:/fl.lutz/FAST/FAST-OAD/FAST-GA/src/fastga
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_chords (ID 68 )
INFO    : Installed bundle fastga.models.load_analysis.loads (ID 121 )
INFO    : Installed bundle fastga.models.aerodynamics.components.cd0_fuselage (ID 19 )
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_distance (ID 69 )
INFO    : Installed bundle fastga.models.propulsion.propulsion (ID 147 )
INFO    : Installed bundle fastga.utils.physics.tests (ID 445 )
INFO    : Installed bundle fastga.models.aerodynamics.components.cd0_vt (ID 25 )
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_efficiency (ID 70 )
INFO    : Installed bundle fastga.models.aerodynamics.components.cd0_other (ID 23 )
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_mac (ID 71 )
INFO    : Installed bundle fastga.

INFO    : Installed bundle fastga.models.aerodynamics.aerodynamics_high_speed (ID 14 )
INFO    : Installed bundle fastga.models.load_analysis.unitary_tests.dummy_engines (ID 125 )
INFO    : Installed bundle fastga.utils (ID 443 )
INFO    : Installed bundle fastga.models.handling_qualities.unitary_tests (ID 414 )
INFO    : Installed bundle fastga.models.aerodynamics.aerodynamics_low_speed (ID 15 )
INFO    : Installed bundle fastga.models.aerodynamics.resources (ID 398 )
INFO    : Installed bundle fastga.models.geometry.geom_components.vt.components.compute_vt_chords (ID 79 )
INFO    : Installed bundle fastga.command.resources (ID 385 )
INFO    : Installed bundle fastga.models.geometry.geom_components.vt.components.compute_vt_mac (ID 80 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.unitary_tests.dummy_engines (ID 196 )
INFO    : Installed bundle fastga.models.geometry.geom_components.vt.components.compute_vt_sweep (ID 81 )
INFO    : Installed bundle fastga.notebooks.tu

INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a1_wing_cg (ID 150 )
INFO    : Installed bundle fastga.models.geometry.geom_components.wing.components (ID 409 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a2_fuselage_cg (ID 151 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.max_cg_ratio (ID 168 )
INFO    : Installed bundle fastga.utils.physics (ID 444 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing.compute_to_rotation_limit (ID 109 )
INFO    : Installed bundle fastga.models.propulsion.fuel_propulsion.basicIC_engine.unitary_tests (ID 424 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a5_landing_gear_cg (ID 154 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.ratio_aft (ID 170 )
INFO    : Installed bundle fastga.models.aerodynamics.external.openvsp.resources (ID 393 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_a

D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\models\aerodynamics\external\vlm\vlm.py:414: RuntimeWarning: invalid value encountered in double_scalars
  htp_e = cl_htp ** 2 / (math.pi * aspect_ratio * cdi_htp)


NL: NLBGS 1 ; 12257370.8 1
NL: NLBGS 2 ; 4452422.16 0.36324447
NL: NLBGS 3 ; 4727882 0.385717465
NL: NLBGS 4 ; 2777158.37 0.226570478
NL: NLBGS 5 ; 795119.577 0.0648686891
NL: NLBGS 6 ; 292014.232 0.0238235619
NL: NLBGS 7 ; 86391.5115 0.00704812743
NL: NLBGS 8 ; 58190.2993 0.00474737203
NL: NLBGS 9 ; 63689.6226 0.00519602642
NL: NLBGS 10 ; 31311.5236 0.0025545057
NL: NLBGS 11 ; 16951.4587 0.00138296042
NL: NLBGS 12 ; 11233.9148 0.000916502813
NL: NLBGS 13 ; 7312.64202 0.000596591401
NL: NLBGS 14 ; 4092.56699 0.000333886202
NL: NLBGS 15 ; 2369.64602 0.000193324169
NL: NLBGS 16 ; 1404.33631 0.000114570761
NL: NLBGS 17 ; 812.178218 6.62603939e-05
NL: NLBGS 18 ; 457.411014 3.73172209e-05
NL: NLBGS 19 ; 259.484238 2.1169649e-05
NL: NLBGS 20 ; 146.986419 1.19916759e-05
NL: NLBGS 21 ; 82.3122033 6.71532293e-06
NL: NLBGS Converged


INFO    : Computation finished after 237.43 seconds
INFO    : Problem outputs written in D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\notebooks\tutorial\workdir\problem_outputs.xml


Let's save these results. We will use them in the next workbook, that shows some post-processing utilities.

In [8]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
Beechcraft_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_Beechcraft_800nm_mda.xml')
shutil.copy(OUTPUT_FILE, Beechcraft_OUTPUT_FILE)

'workdir\\problem_outputs_Beechcraft_800nm_mda.xml'

The `variable-viewer` provides a way to inspect the content of the XML file. The dropdown lists above the table allow to filter the displayed variable.

In [9]:
api_cs25.variable_viewer(OUTPUT_FILE)

In particular, you may inspect the `data:handling_qualities:static_margin` variable.
You will see that its value is slightly negative, which means that the current aircraft configuration is estimated as unstable. We will see later how we can correct this point.

### Beechcraft for 1000 nm range
Here we run an MDA but we change one of the Top Level Aircraft Requirement (TLAR): the range. We choose a 1000 nm range instead of 800 nm for the Beechcraft. For that we use the `VariableViewer` tool on the input file to change the range (do not forget to save!). Just like this:
![variable_viewer](./img/variable_viewer_change_range.gif)

In [10]:
INPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_inputs.xml')
api_cs25.variable_viewer(INPUT_FILE)

Now that the range has been changed, we run again the MDA.

In [11]:
eval_problem = api_cs25.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

INFO    : Loading bundles from D:/fl.lutz/FAST/FAST-OAD/FAST-GA/src/fastga
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_chords (ID 68 )
INFO    : Installed bundle fastga.models.load_analysis.loads (ID 121 )
INFO    : Installed bundle fastga.models.aerodynamics.components.cd0_fuselage (ID 19 )
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_distance (ID 69 )
INFO    : Installed bundle fastga.models.propulsion.propulsion (ID 147 )
INFO    : Installed bundle fastga.utils.physics.tests (ID 445 )
INFO    : Installed bundle fastga.models.aerodynamics.components.cd0_vt (ID 25 )
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_efficiency (ID 70 )
INFO    : Installed bundle fastga.models.aerodynamics.components.cd0_other (ID 23 )
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_mac (ID 71 )
INFO    : Installed bundle fastga.

INFO    : Installed bundle fastga.models.aerodynamics.aerodynamics_high_speed (ID 14 )
INFO    : Installed bundle fastga.models.load_analysis.unitary_tests.dummy_engines (ID 125 )
INFO    : Installed bundle fastga.utils (ID 443 )
INFO    : Installed bundle fastga.models.handling_qualities.unitary_tests (ID 414 )
INFO    : Installed bundle fastga.models.aerodynamics.aerodynamics_low_speed (ID 15 )
INFO    : Installed bundle fastga.models.aerodynamics.resources (ID 398 )
INFO    : Installed bundle fastga.models.geometry.geom_components.vt.components.compute_vt_chords (ID 79 )
INFO    : Installed bundle fastga.command.resources (ID 385 )
INFO    : Installed bundle fastga.models.geometry.geom_components.vt.components.compute_vt_mac (ID 80 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.unitary_tests.dummy_engines (ID 196 )
INFO    : Installed bundle fastga.models.geometry.geom_components.vt.components.compute_vt_sweep (ID 81 )
INFO    : Installed bundle fastga.notebooks.tu

INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a1_wing_cg (ID 150 )
INFO    : Installed bundle fastga.models.geometry.geom_components.wing.components (ID 409 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a2_fuselage_cg (ID 151 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.max_cg_ratio (ID 168 )
INFO    : Installed bundle fastga.utils.physics (ID 444 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing.compute_to_rotation_limit (ID 109 )
INFO    : Installed bundle fastga.models.propulsion.fuel_propulsion.basicIC_engine.unitary_tests (ID 424 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a5_landing_gear_cg (ID 154 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.ratio_aft (ID 170 )
INFO    : Installed bundle fastga.models.aerodynamics.external.openvsp.resources (ID 393 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_a

NL: NLBGS 1 ; 12304844.4 1
NL: NLBGS 2 ; 4452422.35 0.361843043
NL: NLBGS 3 ; 4763096.49 0.387091159
NL: NLBGS 4 ; 2803369.43 0.227826484
NL: NLBGS 5 ; 908975.132 0.0738713226
NL: NLBGS 6 ; 319948.51 0.026001833
NL: NLBGS 7 ; 105963.129 0.00861149684
NL: NLBGS 8 ; 70378.0111 0.00571953685
NL: NLBGS 9 ; 79523.997 0.0064628202
NL: NLBGS 10 ; 41009.8546 0.00333282187
NL: NLBGS 11 ; 22434.6688 0.00182323872
NL: NLBGS 12 ; 14906.3009 0.00121141726
NL: NLBGS 13 ; 10152.7425 0.000825101252
NL: NLBGS 14 ; 5864.01798 0.000476561733
NL: NLBGS 15 ; 3478.46344 0.000282690567
NL: NLBGS 16 ; 2129.74786 0.000173082063
NL: NLBGS 17 ; 1282.1232 0.00010419662
NL: NLBGS 18 ; 753.027516 6.11976463e-05
NL: NLBGS 19 ; 444.542015 3.61273983e-05
NL: NLBGS 20 ; 263.35331 2.14024088e-05
NL: NLBGS 21 ; 154.877817 1.25867351e-05
NL: NLBGS 22 ; 90.5738042 7.36082482e-06
NL: NLBGS Converged


INFO    : Computation finished after 146.23 seconds
INFO    : Problem outputs written in D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\notebooks\tutorial\workdir\problem_outputs.xml


Let's save again these new results, for post-processing them in next notebook.

In [12]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
Beechcraft_1000nm_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_Beechcraft_1000nm_mda.xml')
shutil.copy(OUTPUT_FILE, Beechcraft_1000nm_OUTPUT_FILE)

'workdir\\problem_outputs_Beechcraft_1000nm_mda.xml'

## 3. Running your first MDO

## Beechcraft 800 nm optimization

As seen earlier, the current aircraft configuration is estimated with a fixed rear length (and thus fuselage length). We will unlock this parameter using a simple optimization problem that will move horizontal tail position in order to minimize the mission consumption. 

The default configuration file defines this optimization problem that aims at:
- minimizing the fuel consumption for the mission (objective),
- with respect to the rear fusellage length (design variables): by extension the horizontal tail to wing distance,
- subject to no additional constraint.

*(This run should take approximately 20 minutes)*

In [13]:
# Set back the inputs from the reference Beechcraft 800 nm
api_cs25.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

INFO    : Loading bundles from D:/fl.lutz/FAST/FAST-OAD/FAST-GA/src/fastga
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_chords (ID 68 )
INFO    : Installed bundle fastga.models.load_analysis.loads (ID 121 )
INFO    : Installed bundle fastga.models.aerodynamics.components.cd0_fuselage (ID 19 )
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_distance (ID 69 )
INFO    : Installed bundle fastga.models.propulsion.propulsion (ID 147 )
INFO    : Installed bundle fastga.utils.physics.tests (ID 445 )
INFO    : Installed bundle fastga.models.aerodynamics.components.cd0_vt (ID 25 )
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_efficiency (ID 70 )
INFO    : Installed bundle fastga.models.aerodynamics.components.cd0_other (ID 23 )
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_mac (ID 71 )
INFO    : Installed bundle fastga.

INFO    : Installed bundle fastga.models.aerodynamics.aerodynamics_high_speed (ID 14 )
INFO    : Installed bundle fastga.models.load_analysis.unitary_tests.dummy_engines (ID 125 )
INFO    : Installed bundle fastga.utils (ID 443 )
INFO    : Installed bundle fastga.models.handling_qualities.unitary_tests (ID 414 )
INFO    : Installed bundle fastga.models.aerodynamics.aerodynamics_low_speed (ID 15 )
INFO    : Installed bundle fastga.models.aerodynamics.resources (ID 398 )
INFO    : Installed bundle fastga.models.geometry.geom_components.vt.components.compute_vt_chords (ID 79 )
INFO    : Installed bundle fastga.command.resources (ID 385 )
INFO    : Installed bundle fastga.models.geometry.geom_components.vt.components.compute_vt_mac (ID 80 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.unitary_tests.dummy_engines (ID 196 )
INFO    : Installed bundle fastga.models.geometry.geom_components.vt.components.compute_vt_sweep (ID 81 )
INFO    : Installed bundle fastga.notebooks.tu

INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a1_wing_cg (ID 150 )
INFO    : Installed bundle fastga.models.geometry.geom_components.wing.components (ID 409 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a2_fuselage_cg (ID 151 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.max_cg_ratio (ID 168 )
INFO    : Installed bundle fastga.utils.physics (ID 444 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing.compute_to_rotation_limit (ID 109 )
INFO    : Installed bundle fastga.models.propulsion.fuel_propulsion.basicIC_engine.unitary_tests (ID 424 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a5_landing_gear_cg (ID 154 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.ratio_aft (ID 170 )
INFO    : Installed bundle fastga.models.aerodynamics.external.openvsp.resources (ID 393 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_a

'D:\\fl.lutz\\FAST\\FAST-OAD\\FAST-GA\\src\\fastga\\notebooks\\tutorial\\workdir\\problem_inputs.xml'

To visualize and edit the optimization problem definition (present in the configuration file .toml) you can use the `optimization_viewer` tool. If design variables or constraints have active bounds they are yellow whereas they are red if they are violated. Modifiying the `Initial Value` will modify the input file defined in the configuration file .toml whereas `Value` corresponds to the value found in the output file defined in the configuration file (here it is the 800 nm MDA run).

In [14]:
api_cs25.optimization_viewer(CONFIGURATION_FILE)

In [15]:
optim_problem = api_cs25.optimize_problem(CONFIGURATION_FILE, overwrite=True)

INFO    : Loading bundles from D:/fl.lutz/FAST/FAST-OAD/FAST-GA/src/fastga
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_chords (ID 68 )
INFO    : Installed bundle fastga.models.load_analysis.loads (ID 121 )
INFO    : Installed bundle fastga.models.aerodynamics.components.cd0_fuselage (ID 19 )
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_distance (ID 69 )
INFO    : Installed bundle fastga.models.propulsion.propulsion (ID 147 )
INFO    : Installed bundle fastga.utils.physics.tests (ID 445 )
INFO    : Installed bundle fastga.models.aerodynamics.components.cd0_vt (ID 25 )
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_efficiency (ID 70 )
INFO    : Installed bundle fastga.models.aerodynamics.components.cd0_other (ID 23 )
INFO    : Installed bundle fastga.models.geometry.geom_components.ht.components.compute_ht_mac (ID 71 )
INFO    : Installed bundle fastga.

INFO    : Installed bundle fastga.models.aerodynamics.aerodynamics_high_speed (ID 14 )
INFO    : Installed bundle fastga.models.load_analysis.unitary_tests.dummy_engines (ID 125 )
INFO    : Installed bundle fastga.utils (ID 443 )
INFO    : Installed bundle fastga.models.handling_qualities.unitary_tests (ID 414 )
INFO    : Installed bundle fastga.models.aerodynamics.aerodynamics_low_speed (ID 15 )
INFO    : Installed bundle fastga.models.aerodynamics.resources (ID 398 )
INFO    : Installed bundle fastga.models.geometry.geom_components.vt.components.compute_vt_chords (ID 79 )
INFO    : Installed bundle fastga.command.resources (ID 385 )
INFO    : Installed bundle fastga.models.geometry.geom_components.vt.components.compute_vt_mac (ID 80 )
INFO    : Installed bundle fastga.models.weight.mass_breakdown.unitary_tests.dummy_engines (ID 196 )
INFO    : Installed bundle fastga.models.geometry.geom_components.vt.components.compute_vt_sweep (ID 81 )
INFO    : Installed bundle fastga.notebooks.tu

INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a1_wing_cg (ID 150 )
INFO    : Installed bundle fastga.models.geometry.geom_components.wing.components (ID 409 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a2_fuselage_cg (ID 151 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.max_cg_ratio (ID 168 )
INFO    : Installed bundle fastga.utils.physics (ID 444 )
INFO    : Installed bundle fastga.models.handling_qualities.tail_sizing.compute_to_rotation_limit (ID 109 )
INFO    : Installed bundle fastga.models.propulsion.fuel_propulsion.basicIC_engine.unitary_tests (ID 424 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_airframe.a5_landing_gear_cg (ID 154 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.ratio_aft (ID 170 )
INFO    : Installed bundle fastga.models.aerodynamics.external.openvsp.resources (ID 393 )
INFO    : Installed bundle fastga.models.weight.cg.cg_components.a_a

NL: NLBGS 1 ; 12257370.8 1
NL: NLBGS 2 ; 4452422.16 0.36324447
NL: NLBGS 3 ; 4727882 0.385717465
NL: NLBGS 4 ; 2777158.37 0.226570478
NL: NLBGS 5 ; 795119.577 0.0648686891
NL: NLBGS 6 ; 292014.232 0.0238235619
NL: NLBGS 7 ; 86391.5115 0.00704812743
NL: NLBGS 8 ; 58190.2993 0.00474737203
NL: NLBGS 9 ; 63689.6226 0.00519602642
NL: NLBGS 10 ; 31311.5236 0.0025545057
NL: NLBGS 11 ; 16951.4587 0.00138296042
NL: NLBGS 12 ; 11233.9148 0.000916502813
NL: NLBGS 13 ; 7312.64202 0.000596591401
NL: NLBGS 14 ; 4092.56699 0.000333886202
NL: NLBGS 15 ; 2369.64602 0.000193324169
NL: NLBGS 16 ; 1404.33631 0.000114570761
NL: NLBGS 17 ; 812.178218 6.62603939e-05
NL: NLBGS 18 ; 457.411014 3.73172209e-05
NL: NLBGS 19 ; 259.484238 2.1169649e-05
NL: NLBGS 20 ; 146.986419 1.19916759e-05
NL: NLBGS 21 ; 82.3122033 6.71532293e-06
NL: NLBGS Converged
NL: NLBGS 1 ; 85.63981 1
NL: NLBGS 2 ; 25.4512696 0.297189702
NL: NLBGS 3 ; 14.2102984 0.165930989
NL: NLBGS 4 ; 7.76222695 0.0906380683
NL: NLBGS 5 ; 4.3536978 0.05

NL: NLBGS 25 ; 0.0716557646 0.000254125265
NL: NLBGS 26 ; 0.038613418 0.000136941461
NL: NLBGS 27 ; 0.0220750651 7.82886315e-05
NL: NLBGS 28 ; 0.0133433603 4.73218727e-05
NL: NLBGS 29 ; 0.00733459624 2.60119506e-05
NL: NLBGS 30 ; 0.00421556534 1.49503904e-05
NL: NLBGS 31 ; 0.00244304844 8.66420635e-06
NL: NLBGS Converged
NL: NLBGS 1 ; 282.882416 1
NL: NLBGS 2 ; 210318.372 743.483371
NL: NLBGS 3 ; 49224.26 174.009614
NL: NLBGS 4 ; 33739.9318 119.27193
NL: NLBGS 5 ; 7254.32854 25.6443247
NL: NLBGS 6 ; 3260.72635 11.5267905
NL: NLBGS 7 ; 3003.23357 10.6165438
NL: NLBGS 8 ; 919.931717 3.25199328
NL: NLBGS 9 ; 514.563081 1.81899988
NL: NLBGS 10 ; 409.030437 1.44593801
NL: NLBGS 11 ; 202.198796 0.714780362
NL: NLBGS 12 ; 107.263634 0.379180987
NL: NLBGS 13 ; 65.6592812 0.232108033
NL: NLBGS 14 ; 37.0602252 0.131009292
NL: NLBGS 15 ; 20.2142539 0.071458149
NL: NLBGS 16 ; 11.517819 0.0407159241
NL: NLBGS 17 ; 6.51046937 0.0230147545
NL: NLBGS 18 ; 3.60518922 0.0127444797
NL: NLBGS 19 ; 2.09435

NL: NLBGS 16 ; 0.288266741 0.0334151576
NL: NLBGS 17 ; 0.210702287 0.0244240806
NL: NLBGS 18 ; 0.111695538 0.0129474665
NL: NLBGS 19 ; 0.0622304938 0.00721360275
NL: NLBGS 20 ; 0.0354016552 0.00410367107
NL: NLBGS 21 ; 0.0210787907 0.00244340054
NL: NLBGS 22 ; 0.011809714 0.00136895242
NL: NLBGS 23 ; 0.00679348188 0.000787483379
NL: NLBGS 24 ; 0.00392688438 0.000455194588
NL: NLBGS 25 ; 0.00226327439 0.00026235309
NL: NLBGS 26 ; 0.00129730747 0.000150380627
NL: NLBGS 27 ; 0.000748558423 8.67710148e-05
NL: NLBGS 28 ; 0.000431451022 5.00127203e-05
NL: NLBGS 29 ; 0.00024837475 2.8790978e-05
NL: NLBGS 30 ; 0.000143026358 1.65792566e-05
NL: NLBGS 31 ; 8.2435833e-05 9.55575499e-06
NL: NLBGS Converged
NL: NLBGS 1 ; 8.61479892 1
NL: NLBGS 2 ; 7178.72725 833.301777
NL: NLBGS 3 ; 1463.41988 169.872784
NL: NLBGS 4 ; 1156.27168 134.21923
NL: NLBGS 5 ; 212.850775 24.7075732
NL: NLBGS 6 ; 82.2552195 9.54812994
NL: NLBGS 7 ; 95.9242393 11.1348205
NL: NLBGS 8 ; 29.031653 3.36997453
NL: NLBGS 9 ; 12.93

INFO    : Computation finished after 1372.33 seconds
INFO    : Problem outputs written in D:\fl.lutz\FAST\FAST-OAD\FAST-GA\src\fastga\notebooks\tutorial\workdir\problem_outputs.xml


Let's save these results:

In [16]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
CeRAS_OPT_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_Beechcraft_800nm_mdo.xml')
shutil.copy(OUTPUT_FILE, CeRAS_OPT_OUTPUT_FILE)

'workdir\\problem_outputs_Beechcraft_800nm_mdo.xml'

The `optimizer_viewer` offers a convenient summary of the optimization result:

In [17]:
api_cs25.optimization_viewer(CONFIGURATION_FILE)

You can use the `VariableViewer` tool to see the optimization results for all variables of the system by loading the .xml output file:

In [18]:
RESULT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
api_cs25.variable_viewer(RESULT_FILE)